In [1]:
import numpy as np
import pandas as pd
import yfinance as yf #pip install yfinance
import datetime as dt
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from twilio.rest import Client
import time, threading


F:\Amar-AIML\Softwares\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
resPeriod = 20
start = dt.datetime(2019,6,1)
now = dt.datetime.now()
pivots = []
counter = 0
dates = []
lastPivot=0
oppStocks=[]
supPivots = []
supCounter = 0
supDates = []
supLastPivot=0
supOppStocks=[]
oppList = []

In [3]:
#stock = input("Enter stock data - ")
interval = input("Enter interval time -")
resPeriod = int(input("Enter Resistance perion -"))

Enter interval time -30M
Enter Resistance perion -20


In [4]:
def oppCheck(currPrice,pivot):
    perDiff = (np.subtract(pivot,currPrice)/pivot)*100
    if(perDiff < 1.5):
        return True
    else:
        return False
    

In [5]:
def send_message(d):
    # Your Account Sid and Auth Token from twilio.com/console
    # DANGER! This is insecure. See http://twil.io/secure
    account_sid = 'AC2109c36452c0f9cd0c63ac90cd21851c'
    auth_token = '0b3c30ed78ef5a636d1b326b32edf8a1'
    client = Client(account_sid, auth_token)
    
    message = client.messages.create(
                                  body=d,
                                  from_='whatsapp:+14155238886',
                                  to='whatsapp:+918976661391'
                              )
    
    print(message.sid)

In [6]:
def plotGraph(df,pivots,dates):
    plt.figure(figsize=(20,10))
    df['High'].plot(label="High")
    timeD = dt.timedelta(days=5)
    for i in range(len(pivots)):
        plt.plot_date([dates[i],dates[i]+timeD],[pivots[i],pivots[i]],linestyle="-",linewidth=2,marker=",")
    plt.show()

In [7]:
def plotSupGraph(df,pivots,dates):
    plt.figure(figsize=(20,10))
    df['Low'].plot(label="Low")
    timeD = dt.timedelta(days=5)
    for i in range(len(pivots)):
        plt.plot_date([dates[i],dates[i]+timeD],[pivots[i],pivots[i]],linestyle="-",linewidth=2,marker=",")
    plt.show()

In [8]:
def findRes(stock,interval,resPeriod):
    counter = 0
    #df = pdr.get_data_yahoo('AXISBANK.NS',start,now)
    res={}
    Range = list(np.zeros(resPeriod,dtype=int))
    dateRange = list(np.zeros(resPeriod,dtype=int))

    df = yf.download(stock, start="2020-06-01",interval = interval)
    for i in df.index:
        currentMax=max(Range,default=0)
        value=round(df["High"][i],2)
        Range=Range[1:resPeriod-1]
        Range.append(value)
        dateRange=dateRange[1:resPeriod-1]
        dateRange.append(i)
        
        if currentMax==max(Range,default=0):
            counter+=1
        else:
            counter=0
        if counter==(resPeriod/2):
            lastPivot = currentMax
            dateloc = Range.index(lastPivot)
            lastDate = dateRange[dateloc]
            pivots.append(lastPivot)
            dates.append(lastDate)
            res[lastDate] = lastPivot
    
    currPrice = round(df.tail(1).Close.item(),2)
    currPivot = pivots[len(pivots)-1]
    chkOpp = oppCheck(currPrice,currPivot)
    #pivots.append(currPrice)
    
    if(chkOpp):
        oppStocks.append(stock)
        if(currPrice < currPivot):
            #st = "Res - "+stock+" ,pivot at - ",currPivot,", curr Price at - ",currPrice
            st = st = "Res - "+stock
            oppList.append(st)
    #plotGraph(df,pivots,dates)  # This method is used to plot the graph - we're not using it for now
    


In [9]:
def findSup(stock,interval,resPeriod):
    #df = pdr.get_data_yahoo('AXISBANK.NS',start,now)
    supCounter=0
    res={}
    Range = list(np.zeros(resPeriod,dtype=int))
    dateRange = list(np.zeros(resPeriod,dtype=int))
    

    df = yf.download(stock, start="2020-06-01",interval = interval)

    for i in df.index:
        currentMin=min(Range,default=0)
        value=round(df["Low"][i],2)
        Range=Range[1:resPeriod-1]
        Range.append(value)
        dateRange=dateRange[1:resPeriod-1]
        dateRange.append(i)
        if currentMin==min(Range,default=0):
            supCounter+=1
        else:
            supCounter=0
        
        if supCounter==(resPeriod/2):
            supLastPivot = currentMin
            dateloc = Range.index(supLastPivot)
            lastDate = dateRange[dateloc]
            supPivots.append(supLastPivot)
            supDates.append(lastDate)
            res[lastDate] = supLastPivot
    
    currPrice = round(df.tail(1).Close.item(),2)
    currPivot = pivots[len(pivots)-1]
    chkOpp = oppCheck(currPivot,currPrice)
    
    if(chkOpp):
        supOppStocks.append(stock)
        if(currPrice > currPivot):
            #st = "Supp - "+stock+" , pivot at - ",currPivot,", curr Price at - ",currPrice
            st = "Supp - "+stock
            oppList.append(st)
    #plotSupGraph(df,pivots,dates)  # This method is used to plot the graph - we're not using it for now

In [10]:
def call_generator():
    stock_data = pd.read_csv(r"C:\Users\abc\Downloads\AI-ML\git\Stock code\stock_data.csv")
    listOfStocks = stock_data['Stocks'].tolist()
    #listOfStocks = ['SBIN.NS']
    oppList = []
    for stock in listOfStocks:
       
        counter = 0
        dates = []
        lastPivot=0
        oppStocks=[]
        supPivots = []
        supCounter = 0
        supDates = []
        supLastPivot=0
        supOppStocks=[]
        try:
            findRes(stock,interval,resPeriod)
            findSup(stock,interval,resPeriod) 
        except Exception as e:
            print(e)
    d = str(oppList)
    send_message(d)

In [11]:
call_generator()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [12]:
pivots

[706.6,
 711.9,
 725.8,
 729.4,
 719.45,
 712.0,
 716.6,
 710.25,
 726.7,
 751.6,
 742.1,
 765.45,
 796.9,
 790.0,
 805.0,
 806.35,
 952.0,
 924.45,
 949.55,
 967.55,
 2081.0,
 2100.0,
 2095.0,
 2132.0,
 2073.9,
 2090.0,
 2064.25,
 2130.9,
 2113.7,
 2301.75,
 2274.35,
 2249.25,
 2244.95,
 2332.05,
 2237.9,
 2232.0,
 2341.25,
 2357.8,
 1538.2,
 1559.25,
 1617.25,
 1616.65,
 1579.4,
 1606.1,
 1646.75,
 1635.0,
 1803.75,
 1772.3,
 1752.0,
 1740.75,
 1863.75,
 1947.7,
 1977.95,
 2198.45,
 2194.0,
 347.95,
 373.0,
 373.75,
 369.1,
 356.75,
 346.15,
 347.85,
 350.85,
 375.2,
 379.8,
 358.3,
 354.45,
 370.85,
 367.6,
 379.0,
 372.4,
 366.5,
 356.75,
 366.3,
 389.3,
 395.65,
 385.6,
 357.5,
 995.0,
 1044.05,
 1043.0,
 1065.8,
 998.95,
 986.45,
 998.8,
 1045.45,
 1057.35,
 1062.6,
 1082.6,
 1111.0,
 1118.7,
 1129.5,
 1079.0,
 1152.85,
 1157.0,
 1143.5,
 1121.8,
 1094.4,
 573.4,
 572.35,
 586.1,
 583.5,
 583.6,
 587.5,
 593.45,
 583.8,
 583.5,
 573.7,
 594.25,
 594.7,
 591.5,
 613.15,
 652.1,
 6

In [13]:
dates

[Timestamp('2020-06-01 10:30:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-03 09:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-08 12:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-09 12:30:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-10 14:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-16 11:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-18 15:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-19 15:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-24 11:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-06-26 15:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-01 09:30:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-02 13:30:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-07 15:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-09 09:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-13 09:30:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-14 09:00:00+0530', tz='Asia/Kolkata'),
 Timestamp('2020-07-16 10:00:00+0530', tz='Asia/Kolkata'

In [14]:
plt.figure(figsize=(20,10))
df.High.plot()
plt.show()

NameError: name 'df' is not defined

<Figure size 1440x720 with 0 Axes>

In [ ]:
send_message("adffad")

# Code to find financial details of the stock

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,8))
sns.regplot(y=pivots,x=np.arange(0,len(pivots)))

In [ ]:
pivots

In [ ]:
supPivots

In [ ]:
master = pd.concat([pd.Series(pivots),pd.Series(supPivots)])

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,8))
sns.regplot(y=master,x=np.arange(0,len(master)))

In [ ]:
argrelextrema(np.array(pivots), np.greater)

In [ ]:
n = np.random.random(12)

In [ ]:
type(n)

In [ ]:
 df = yf.download('AXISBANK.NS', start="2020-06-01",interval = '30m')

In [ ]:
df.tail(20)